# **RAG with Semantic Kernal using Astra DB**

This an example of building a Retrieval Augmented Generation (RAG) application efficiently by configuring Semantic Kernal using Astra DB MemoryStore. You just need to folllow and execute those steps below.




### Install libraries


In [1]:
%pip install -q semantic-kernel
%pip install -q ragstack-ai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import Modules

In [ ]:
import semantic_kernel.connectors.ai.open_ai as sk_oai
from semantic_kernel.prompt_template.input_variable import InputVariable

### Load ENV Variables




In [2]:
from dotenv import dotenv_values
env = dotenv_values(".env")

### Initialize Semantic Kernel and Add Chat & Embedding services

In [26]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    OpenAITextEmbedding
)

CHAT_COMPLETION_MODEL = "gpt-3.5-turbo"
EMBEDDING_MODEL = "text-embedding-ada-002"

# Semantic Kernel
kernel = sk.Kernel()

#A dd OpenAI chat_completion service
kernel.add_service(
    OpenAIChatCompletion(
        service_id="completion",
        ai_model_id=CHAT_COMPLETION_MODEL,
        api_key=env["OPEN_AI_API_KEY"]
    )
)
print("Added OpenAI Chat Service")

# Add OpenAI text_embedding service
kernel.add_service(
    OpenAITextEmbedding(
        service_id="embedding",
        ai_model_id="text-embedding-ada-002",
        api_key=env["OPEN_AI_API_KEY"]
    )
)

print("Added OpenAI Embedding Service")

Added OpenAI Chat Service
Added OpenAI Embedding Service


# **Register Astra DB memory in Semantic kernal**

In [ ]:
from semantic_kernel.connectors.memory.astradb import AstraDBMemoryStore
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.memory_store_base import MemoryStoreBase
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin

# Create Astra memory store

store = AstraDBMemoryStore(env["ASTRA_DB_TOKEN"], 
                           env["ASTRA_DB_ID"], 
                           env["ASTRA_REGION"], 
                           env["KEYSPACE"], 
                           int(env["EMBEDDING_DIMENSION"]), 
                           env["SIMILARITY"])


# Register Astra memory in Semantic Kernal Memory
memory = SemanticTextMemory(storage=store, embeddings_generator=kernel.get_service("embedding"))
kernel.import_plugin_from_object(TextMemoryPlugin(memory), "TextMemoryPluginACDB")

### Document loading and embedding

In [24]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def find_pdfs(directory):
    pdf_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files
articles = find_pdfs(".")

print("Saving vectors to ",env["COLLECTION_NAME"] )

for i, article in enumerate(articles):
    print("Processing ", article)
    loader = PyPDFLoader(article)
    pages = loader.load_and_split()
    s = str(await memory._embeddings_generator.generate_embeddings("bbbbbb"))
    print(s)
    for j, page in enumerate(pages):
        print(page)
        await memory.save_information(
                        id= "{0}-{1}".format(i,j),
                        collection=env["COLLECTION_NAME"],
                        text=page.page_content
        )

Multiple definitions in dictionary at byte 0x3d9cb for key /Rotate
Multiple definitions in dictionary at byte 0x3da82 for key /Rotate


Saving vectors to  documents
Processing  ./data/2403.06995.pdf
[[-0.02716577 -0.02484674  0.01183671 ... -0.01191953 -0.00112414
   0.01370022]]
page_content='arXiv:2403.06995v1  [cs.AI]  3 Mar 2024EXACT ALGORITHMS AND HEURISTICS FOR CAPACITATED\nCOVERING SALESMAN PROBLEMS\nLucas Porto Maziero, Fábio Luiz Usberti, Celso Cavellucci\nInstitute of Computing\nUniversidade Estadual de Campinas\nCampinas\n{lucas.maziero, fusberti, celsocv}@ic.unicamp.br\nABSTRACT\nThis paper introduces the Capacitated Covering Salesman Pr oblem (CCSP), approaching the notion\nof service by coverage in capacitated vehicle routing probl ems. In CCSP, locations where vehicles\ncan transit are provided, some of which have customers with d emands. The objective is to ser-\nvice customers through a ﬂeet of vehicles based in a depot, mi nimizing the total distance traversed\nby the vehicles. CCSP is unique in the sense that customers, t o be serviced, do not need to be\nvisited by a vehicle. Instead, they can be se

#### Chat functionality from OpenAI chat model

In [32]:
import semantic_kernel.connectors.ai.open_ai as sk_oai
from semantic_kernel.prompt_template.input_variable import InputVariable

# Simple Prompt
prompt = """
    You a computer scientist researcher summarizing articles recently publsihed. Use only the articles described on the article context to answer questions in a summary form
    article context: {{$db_content}}
    
    Also, answer the question in Pirate English
    
    Chatbot:"""

# configure chat settings for chat_completion model
execution_settings = sk_oai.OpenAITextPromptExecutionSettings(
   service_id="completion",
    ai_model_id=CHAT_COMPLETION_MODEL,
    max_tokens=500,
    temperature=0.0,
    top_p=0.5
)

chat_prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="grounded_response",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="db_content", description="database content", is_required=True),
        InputVariable(name="users_query", description="user input", is_required=True),
    ],
    execution_settings=execution_settings
)

chat = kernel.create_function_from_prompt(
 prompt=prompt,
 function_name= "ChatGPTFunc2", plugin_name="chatGPTPlugin2", prompt_template_config=chat_prompt_template_config
)


Overwriting function "ChatGPTFunc2" in collection


### Execute RAG flow

In [33]:
# query whatever you want and it respond with as per given context
users_query = "summarize the Phase autoencoder for limit-cycle oscillators is in simple terms"

result = await memory.search(env["COLLECTION_NAME"], users_query);
completions_result = await kernel.invoke(chat, sk.KernelArguments(users_query=users_query, db_content=result[0].text))
print(completions_result)


Arrr matey! The article be discussin' a phase autoencoder for limit-cycle oscillators, which be a machine-learnin' method that can evaluate the asymptotic phase and phase sensitivity function of an oscillator only from time-series data. This autoencoder can also reconstruct the oscillator state from the phase value. It be useful for synchronizin' two oscillators without needin' their mathematical models. Spontaneous rhythmic phenomena, like heartbeats and brain waves, can be modeled as limit-cycle oscillators, and this phase autoencoder helps understand their synchronization dynamics. Aye, it be a handy tool for data-driven control of limit-cycle oscillators!
